# Utilities
## preparing Data and TraingVisualization
**助教给的demo code 一眼难尽啊导致我也写的烂...  
HW1和HW2代码可读程度完全不是一个等级**

In [90]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn

#dataPreprocess
import csv
import numpy as np
import pandas as pd 
import os
import random

#plotting Tools
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#seed
myseed = 420613
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)
device_count = torch.cuda.device_count()
print(f"Number of CUDA devices: {device_count}")


Number of CUDA devices: 1


# Some Utilites

In [91]:
def get_device():
    '''geting device if cuda is available'''
    return 'cuda:0' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record,title = ''):
    '''plot learning curve of your DNN()(train&dev loss)'''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train'])//len(loss_record['dev'])]
    figure(figsize=(6,4))
    plt.plot(x_1,loss_record['train'],c = 'tab:red', label = 'train')
    plt.plot(x_2,loss_record['dev'],c = 'tab:cyan', label = 'dev')
    plt.ylim(0.0,5.)
    plt.xlabel('TrainSteps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()
    
def plot_pred(dv_set, model, device, lim = 35., preds = None, targets = None):
    '''plot predition of your DNN'''
    if preds is None or targets is None:
        model.eval()#set model to eval mode
        preds, targets = [], []
        for x, y in dv_set:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                preds.append(pred.detach().cpu())
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim = 0).numpy()
        targets = torch.cat(targets, dim = 0).numpy()
        
    figure(figsize=(5,5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    plt.plot([-0.2,lim], [-0.2, lim], c = 'b')
    plt.xlim(-0.2,lim)
    plt.ylim(-0.2,lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth with Prediction')
    plt.show()

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides) 

**下面的Helper函数预处理每个语句的MFCC原始特征训练数据，由于一个音素可以跨越几个帧且对过去和未来的帧具有依赖性  
我们对相邻的音素进行训练，以便达到更高的准确性。连接过去和未来的k帧（总共2k+1 = n帧），我们预测中心帧。  
可以随意修改预处理函数但不要删除任何帧**

In [92]:
#由于文件打开不了，我们将文件读一个出来看看大小数据类型分别是什么样子的
feat = torch.load('./dataset/libriphone/feat/train/19-198-0008.pt')
cur_len = len(feat)
print(cur_len)
print(feat.shape)
print(type(feat))



284
torch.Size([284, 39])
<class 'torch.Tensor'>


In [93]:
def load_feature(path):
    '''从文件中加载特征'''
    feat = torch.load(path)
    return feat
# def shift(x, n):
#     '''
#     数据扩展，特征拼接
#     将输入张量x向左或向右移动n个距离用边界值填充移动后的空缺位置。
#     x：张量 n：移动距离，负为向右移动，反之向左移动
#     '''
#     if n < 0 :
#         right = x[:n]
#         left = x[0].repeat(-n, 1)
#     elif n > 0:
#         right = x[-1].repeat(n, 1)
#         left = x[n:]
#     else:
#         return x
#     return torch.cat((left,right), dim = 0)
def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
        # print(left)
        # print(right)
        # print("x[:-1]:",x[:-1])

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)
# x = torch.tensor([[1,2,3],
#                   [4,5,6],
#                   [7,8,9],
#                   [10,11,12]])
# y = shift(x,1)
# z = shift(x,-1)
# print("y:{}".format(y))
# print("z:{}".format(z))

def concat_feat(x, concate_n):
    '''
    x：张量，为直接的特征
    concate_n:
    '''
    assert concate_n % 2 == 1, '拼接数为偶数'
    if concate_n < 2 :
        return x
    seq_len, featrue_dim = x.size(0), x.size(1)
    x = x.repeat(1,concate_n)
    x = x.view(seq_len, concate_n, featrue_dim).permute(1, 0, 2)
    mid = (concate_n // 2)
    for r_idx in range(1, mid + 1):
        #由中心张量复制出来的两侧张量都向中间靠拢，每离中心向量一格距离则移动一格
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)              #右侧张量向左移动
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)             #左侧张量向右移动
        
    return x.permute(1, 0, 2).view(seq_len, featrue_dim*concate_n)   #将处理好的张量拼接回原来的样子

def train_label(mode,split,phone_path):
    '''
    split:模型训练不同阶段train/val/test
    phone_path:文件路径
    
    建立起train_data文件名和label之间的映射，返回字典
    '''
    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
    return label_dict

def split_train_val(split, phone_path, seed, train_ratio):
    '''
    split:模型训练不同阶段train/val/test
    phone_path:meta文件父路径
    seed：随机种子
    train_ratio：训练验证集划分比
    
    将train_data划分为训练集和验证集
    '''
    if split == 'train' or split == 'val' :
        usage_list = open(os.path.join(phone_path,'train_split.txt')).readlines()
        random.seed(seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        
        print(percent)
        
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')
    
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(41) + 
          ', number of utterances for ' + split + ': ' + str(len(usage_list)))
    
    return usage_list

def process_feature(split, 
                    feat_dir, 
                    concat_nframes, 
                    label_dict, 
                    usage_list):
    '''
    对每个语音（文件名）对应的特征进行根据usagelist来读取对应文件特征，并根据给出的concat_nframes来
    判断需要对扩展出几个帧,主要是一些比较杂的判断和处理，比如创建空张量、
    '''
    mode = 'train' if split == 'train' or split == 'val' else 'test'
    #1.创建空张量便于将不同文件中的特征输入到张量中
    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)                              #
    if mode != 'test' :
        y = torch.empty(max_len,dtype=torch.long)
    
    #2.读取文件
    #根据usage_list来选择train/val/test对应的特征，cur_len是每个特征文件的特征向量个数
    idx = 0
    for i,fname in tqdm(enumerate(usage_list)):
        feat = load_feature(os.path.join(feat_dir,mode,f'{fname}.pt'))
        
        print("从文件{}中加载出来的特征形状，类型：".format(fname),feat.shape,type(feat))#一个文件中
        
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])
            
        X[idx: idx + cur_len,:] = feat
        if mode != 'test':
            y[idx: idx + cur_len] = label
        
        idx += cur_len
        
    #冗余0张量空间消除，同时做一些不同mode下的判断处理
    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]
        
    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X,y
    else:
        return X,None                #为了增强代码鲁棒性，在测试集情况下我们将标签读取值同样返回为None这样方便判断
    
def preprocess_data(split,           #mode：train/val/test
                    feat_dir,        #特征父文件夹
                    phone_path,      #文件名父文件夹
                    concat_nframes,  #拼接数量
                    train_ratio,     #训练集验证集划分
                    train_val_seed): #随机种子
    '''
    包含处理数据集输入以及张量拼接的一系列操作，以及train/val划分
    '''
    class_num = 41
    mode = 'train' if split == 'train' or split == 'val' else 'test'          #确定计算模式
    
    label_dict = train_label(mode=mode,split=split, phone_path=phone_path)              #建立起文件名和标签之间的联系
    
    usage_list = split_train_val(split=split,                                 #根据模式返回打乱后的数据集/验证集
                                 phone_path=phone_path, 
                                 seed=train_val_seed,
                                 train_ratio=train_ratio)
    return process_feature(split = split,                                       #根据文件名提取特征，以及concate
                    feat_dir = feat_dir,                                      #feat操作，
                    concat_nframes = concat_nframes, 
                    label_dict = label_dict, 
                    usage_list = usage_list)

**repeat函数参数个数必须 >= 给定张量的维度个数，如果给出的参数数量m大于给定的张量唯独数量n，则
优先将 m 个参数中后 n 个参数匹配到张量的维度上进行repeat**
**在理解了repeat的基础上就很容易理解上面几个函数的作用**

```python
view()函数在面对多个参数时的行为也有些难以理解，我们按照将张量展平成1维向量之后再对张量进行处理
x:
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])
展平的数据: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
新张量的形状是 (2, 2, 3)，即 2 个大小为 (2, 3) 的小张量。
按照展平的数据顺序依次填充新张量。
y:
tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],
        [[ 7,  8,  9],
         [10, 11, 12]]])
repeat函数举例
x:
tensor([[ 1,  2],  
        [ 3,  4],  
        [ 5,  6],  
        [ 7,  8],  
        [ 9, 10]])  
x = x.repeat(1, concat_n)
x:  
tensor([[ 1,  2,  1,  2,  1,  2],  
        [ 3,  4,  3,  4,  3,  4],  
        [ 5,  6,  5,  6,  5,  6],  
        [ 7,  8,  7,  8,  7,  8],  
        [ 9, 10,  9, 10,  9, 10]])  
seq_len = 5, concat_n = 3, feature_dim = 2
5个大小为3*2的二维向量，相当于把每一列重复的数值都放在了一个二维向量里
再通过permute方法将维度进行重组，调整第一个维度和第二个维度的位置就可以做到
将三个重复的向量进行拆分

x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2)
x:
tensor([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10]],

        [[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10]],

        [[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10]]])

concate_feature中的后半部分操作：
x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

shift(x[mid + 1], 1):
tensor([[ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [ 9, 10]])

shift(x[mid - 1], -1):
tensor([[ 1,  2],
        [ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8]])

最终结果:
tensor([[[ 1,  2],
         [ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8]],

        [[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10]],

        [[ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10],
         [ 9, 10]]])
result = x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)
result:
！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！重点！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
有没有看出经过操作之后的张量的每一行怎么样了，从第二行开始，我们的位移操作实际上是将与中心音素相差时刻数的音素
进行偏移最后拼接起来这样既将前后时序的音素拼接起来了，又将音素特征的大小从39扩充到了39*concat_nframes的大小
既使得网络能够考虑到上下文，又增强了模型的输入大小。这里shift + concate的操作真是太妙了。
tensor([[ 1,  2,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9, 10],
        [ 7,  8,  9, 10,  9, 10]])

```

# 数据预处理核心
！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！重点！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！  
有没有看出经过操作之后的张量的每一行怎么样了，从第二行开始，我们的位移操作实际上是将与中心音素相差时刻数的音素  
进行偏移最后拼接起来这样既将前后时序的音素拼接起来了，又将音素特征的大小从39扩充到了39*concat_nframes的大小  
既使得网络能够考虑到上下文，又增强了模型的输入大小。这里shift + concate的操作真是太妙了。  
tensor([  
[ 1,  2,  1,  2,  3,  4],  
[ 1,  2,  3,  4,  5,  6],  
[ 3,  4,  5,  6,  7,  8],  
[ 5,  6,  7,  8,  9, 10],  
[ 7,  8,  9, 10,  9, 10]  
])  


# Dataset & DataLoader
**对于示例代码中的代码结构有些粗糙了，将许多步骤分离。应该将数据处理部分的操作都放入继承并实现Dataset的数据类中  
这样对于输入文件路径给dataLoader就能够直接返回迭代器。 避免分离处理不同的代码组成部分。  
下面将实现把数据文件的读入、预处理等操作集合到自己实现的dataset数据集中**

In [94]:
class SpeechDataset(Dataset):
    def __init__(self, 
                split,
                feat_dir, 
                phone_path, 
                concat_nframes, 
                train_ratio, 
                train_val_seed):
        
        self.data, self.label = preprocess_data(split=split,
                                            feat_dir=feat_dir,
                                            phone_path=phone_path,
                                           concat_nframes=concat_nframes,
                                            train_ratio=train_ratio,
                                            train_val_seed=train_val_seed)
        if split == 'train' or split == 'val':
            assert self.label != None, 'label为空'                    #断言（若为训练/验证集）label不为空
        else :
            assert self.label == None, 'label不为空，存在错误'
        
            
    def __getitem__(self,index):
        if self.label is not None:
            return self.data[index], self.label[index]
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    
#DataLoader   
def PrepDataloader(split,
                   batch_size,
                   feat_dir, 
                   phone_path, 
                   concat_nframes,
                   n_jobs = 0,
                   train_ratio = 0.8, 
                   train_val_seed = 1337):
    dataset = SpeechDataset(split=split,
                            feat_dir=feat_dir,
                            phone_path=phone_path,
                            concat_nframes=concat_nframes,
                            train_ratio = train_ratio,
                            train_val_seed = train_val_seed)
    
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=(split == 'train'),
                            num_workers=n_jobs,
                            pin_memory=True,
                            drop_last=False)
    return dataloader

# NeuralNetwork
**该分类问题一共41个标签类别分别代表41个音素，在模型的输出层设计上需要进行考虑。  
这里的模型设计可以使用RNN来处理语音特征，但具体怎么使用还不太清楚  
初始模型非常简单，就是两个全连接层外加一个列表表达式中全连接层的堆叠**

In [95]:
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock,self).__init__()
        
        self.block = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim,output_dim),
            nn.LeakyReLU(),
        )
    def forward(self,x):
        x = self.block(x)
        return x

class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier,self).__init__()
        
        self.net = nn.Sequential(
            
            BasicBlock(input_dim, hidden_dim),
            
            nn.Dropout(0.3),
            ##列表生成式，由于不需要使用序列中的序列值。所以用_来表示
            *[BasicBlock(hidden_dim,hidden_dim) for _ in range(hidden_layers)],
            
            nn.BatchNorm1d(hidden_dim),
            
            nn.Linear(hidden_dim, output_dim)
        )
        self.criterion = nn.CrossEntropyLoss()
    def forward(self,x):
        x = self.net(x)
        return x
    def cal_loss(self, pred, labels, L2_lambda = 0.0):
        crossEntropyLoss = self.criterion(pred, labels)
        # L2 regularization：
        # l2_reg = 0.0
        # if L2_lambda > 0.0:
        #     for parm in self.parameters():
        #         l2_reg += torch.sum(parm**2)
        return crossEntropyLoss #+ l2_reg

# Train/Dev/Test
## Training
**经验总结**
* 1.在处理`loss`（如比较大小，计算平均值，可视化）之前使用变量存储从计算图中分离出来并转化为标量（numpy）的loss值
* 2.在分类问题中要提取预测结果中得分最大的类别常用：`_,pred_maxProb_class = torch.max(pred, 1)`

In [96]:
import torch

#用于保存模型，如果每次不全部运行所有代码由于best_acc的全局性可以保证保存的模型在内存不被清除之前是全局最优的
def train(tr_set, dev_set, model, config, device):
    
    #setup optimizer
    optimizer = getattr(torch.optim,config['optimizer'])(
        model.parameters(),**config['optim_hparas'])
    
    n_epochs = config['n_epochs']
#     min_crossEntropy = 10000                            #在使用早停后可以考虑记录min_crossEntropy的最小值以及
#     min_acc = 0                                         #
    loss_record = {'train':[], 'dev':[]}                  #记录训练损失并且由于是分类任务还需要计算ACC
    acc_record = {'train':[], 'dev':[]}                   #将损失在训练过程可视化有利于模型调参和发现问题
    best_acc = 0                                   #这里看到示例代码中epoch数量比较小，所以就不设置早停了
    
    #training
    epoch = 0
    for epoch in range(n_epochs):
        model.train()
        epoch_acc = 0
        epoch_train_loss = 0
        set_size = 0
        for x,y in tr_set:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            n_epochs
            pred = model(x)
            
#             print(f'inputs device: {x.device}, labels device: {y.device}')
#             print(f'model device: {next(model.parameters()).device}')
            
            batch_cross_entropy_loss = model.cal_loss(pred, y, config['L2_lambda'])
            batch_cross_entropy_loss.backward()
            optimizer.step()
            #dim (int) – the dimension to reduce.对于具有多个参数的tensor.max()第二个返回值是index
            
            _,pred_maxProb_class = torch.max(pred, 1)                      #获得每个预测张量中最大概率值的标签

            batch_acc = (pred_maxProb_class.detach() == y.detach()).sum().item()       #计算单个batch的acc
            epoch_acc += batch_acc
            epoch_train_loss += (batch_cross_entropy_loss.detach().cpu().item())*len(x)#len无需将张量分离出来
            set_size += len(x)
            loss_record['train'].append(batch_cross_entropy_loss.detach().cpu().item())#记录训练损失
            acc_record['train'].append(epoch_acc/len(x))
            
        print(len(tr_set.dataset) == set_size)
        
        epoch_acc /= len(tr_set.dataset)                   #熟悉这种由batchloss*batchsize->total/trainsize的过程
        assert epoch_acc <= 1, 'acc数值异常'
        epoch_train_loss /= len(tr_set.dataset)                  #其实是可以不用计算每个训练集epoch的loss的 但是个好习惯
        
        #validation 
        if len(dev_set.dataset) > 0:
            dev_loss, dev_acc = dev(dev_set,model,device)
            loss_record['dev'].append(dev_loss)
            acc_record['dev'].append(dev_acc)
            #visualization(print loss&acc info)
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Dev Acc: {:3.6f} loss: {:3.6f}'
                  .format(epoch + 1, n_epochs, epoch_acc, epoch_train_loss, dev_acc, dev_loss))
            
            #save model everytime acc gets smaller.
            if dev_acc > best_acc: 
                best_acc = dev_acc
                torch.save(model.state_dict(),config['save_path'])
                print('saving model with the acc of {:.3f}'.format(dev_acc))
        else :
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, n_epochs, epoch_acc, epoch_train_loss))
            
            
    
    # if not validating, save the last epoch，(when small dataset)
    if len(dev_set.dataset) == 0:
        torch.save(model.state_dict(), config['save_path'])
        print('saving model at last epoch') 
    return best_acc, loss_record, acc_record

关于多参数的torch.max用法
```python
a = torch.randn(4, 4)
print(a)
torch.max(a, 1) dim:the dim to reduce。由于预测结果的输出是行向量，我们需要得到行上的最大值，假设我们找到了每行的最大值，此时得到了4*1的张量，那么reduce的dim就变成了列，我们找行上的最大值其实是列与列之间进行的比较。所以参数dim=1
tensor([[-0.6971,  0.3609, -0.3830,  0.3976],
        [ 0.3982, -0.4047, -0.7660, -0.6009],
        [-1.7744,  0.7950,  1.1826, -0.3340],
        [ 0.4587, -1.7904, -1.5404, -0.2847]])
torch.return_types.max(
values=tensor([0.4587, 0.7950, 1.1826, 0.3976]),
indices=tensor([3, 2, 2, 0]))
```
# Testing save pred

# Validation

In [97]:
'''
    我们需要的是整个验证集的交叉熵损失，但对于每个batch取了平均，所以
'''
def dev(dev_set, model, device):
    total_loss = 0
    total_acc = 0
    model.eval()
    for x, y in dev_set:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            dev_pred = model(x)
            batch_loss = model.cal_loss(dev_pred,y,config['L2_lambda'])
        _,max_predProb_class = torch.max(dev_pred, 1)
        
        total_loss += batch_loss*len(x)
        total_acc += (max_predProb_class.detach() == y.detach()).sum().item()
        #这里total_acc表示所有验证样本中TP的数量即TruePositive，处理方式与dev_loss相同
        
    dev_loss = total_loss/len(dev_set.dataset)
    dev_acc = total_acc/len(dev_set.dataset)
    return dev_loss, dev_acc
        

In [98]:
def test(test_set, model, device):
    model.eval()
    preds = []
    for x in test_set:
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            _,pred_class = torch.max(pred,1) #得到概率最大得分对应的类别转化为对应index
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds,dim = 0).numpy()
    return preds

def save_pred(preds, file):
    '''Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer.writerow(['Id', 'Class'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

```python
#在哪个维度上拼接则保持其他维度不变
>>> x = torch.randn(2, 3)
>>> x
tensor([[ 0.6580, -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497]])
>>> torch.cat((x, x, x), 0)
tensor([[ 0.6580, -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497],
        [ 0.6580, -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497],
        [ 0.6580, -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497]])
>>> torch.cat((x, x, x), 1)
tensor([[ 0.6580, -1.0969, -0.4614,  0.6580, -1.0969, -0.4614,  0.6580,  -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497, -0.1034, -0.5790,  0.1497, -0.1034,  -0.5790,  0.1497]])
```

# Setup Hyper-parameters
`config` contains hyper-parameters for training and the path to save your model 

In [99]:
device = get_device()
print(device)
config = {
    'n_epochs':5,
    'batch_size':512,
    'optimizer':'Adam',
    'optim_hparas':{
        'lr':0.0001,
#         'momentum':0.9,
        #'weight_decay': 5e-4,
    },
    'early_stop':20,
    'L2_lambda':5e-4,
    'save_path':'models/model.pth',
    'feat_dir':'./dataset/libriphone/feat/',
    'phone_path':'./dataset/libriphone/'
}
input_dim = 39
concat_nframes = 5


cuda:0


In [100]:

train_data = PrepDataloader('train', config['batch_size'], 
                            config['feat_dir'], 
                            config['phone_path'], 
                            concat_nframes,
                            n_jobs=0)
dev_data = PrepDataloader('val', config['batch_size']
                          ,config['feat_dir'],
                          config['phone_path'],
                          concat_nframes,
                          n_jobs=0)

test_data = PrepDataloader('test', config['batch_size'],
                           config['feat_dir'],
                           config['phone_path'],
                           concat_nframes,
                           n_jobs=0) 

3428
[Dataset] - # phone classes: 41, number of utterances for train: 3428


125it [00:00, 625.74it/s]

从文件8465-246942-0027中加载出来的特征形状，类型： torch.Size([936, 39]) <class 'torch.Tensor'>
从文件198-209-0030中加载出来的特征形状，类型： torch.Size([653, 39]) <class 'torch.Tensor'>
从文件8770-295462-0040中加载出来的特征形状，类型： torch.Size([598, 39]) <class 'torch.Tensor'>
从文件3526-176653-0038中加载出来的特征形状，类型： torch.Size([485, 39]) <class 'torch.Tensor'>
从文件8747-293952-0030中加载出来的特征形状，类型： torch.Size([248, 39]) <class 'torch.Tensor'>
从文件4406-16883-0035中加载出来的特征形状，类型： torch.Size([503, 39]) <class 'torch.Tensor'>
从文件60-121082-0044中加载出来的特征形状，类型： torch.Size([613, 39]) <class 'torch.Tensor'>
从文件2182-181183-0010中加载出来的特征形状，类型： torch.Size([367, 39]) <class 'torch.Tensor'>
从文件8629-261140-0019中加载出来的特征形状，类型： torch.Size([318, 39]) <class 'torch.Tensor'>
从文件2952-407-0019中加载出来的特征形状，类型： torch.Size([631, 39]) <class 'torch.Tensor'>
从文件311-124404-0029中加载出来的特征形状，类型： torch.Size([429, 39]) <class 'torch.Tensor'>
从文件6476-57446-0047中加载出来的特征形状，类型： torch.Size([572, 39]) <class 'torch.Tensor'>
从文件1594-135914-0091中加载出来的特征形状，类型： torch.Size([897, 39]) <class '

194it [00:00, 651.69it/s]

从文件211-122425-0057中加载出来的特征形状，类型： torch.Size([412, 39]) <class 'torch.Tensor'>
从文件78-368-0006中加载出来的特征形状，类型： torch.Size([865, 39]) <class 'torch.Tensor'>
从文件1926-147987-0026中加载出来的特征形状，类型： torch.Size([666, 39]) <class 'torch.Tensor'>
从文件8063-274116-0035中加载出来的特征形状，类型： torch.Size([205, 39]) <class 'torch.Tensor'>
从文件839-130898-0018中加载出来的特征形状，类型： torch.Size([767, 39]) <class 'torch.Tensor'>
从文件3947-13262-0016中加载出来的特征形状，类型： torch.Size([482, 39]) <class 'torch.Tensor'>
从文件2007-132570-0032中加载出来的特征形状，类型： torch.Size([483, 39]) <class 'torch.Tensor'>
从文件3830-12530-0045中加载出来的特征形状，类型： torch.Size([763, 39]) <class 'torch.Tensor'>
从文件150-132655-0012中加载出来的特征形状，类型： torch.Size([811, 39]) <class 'torch.Tensor'>
从文件6531-61334-0076中加载出来的特征形状，类型： torch.Size([345, 39]) <class 'torch.Tensor'>
从文件5049-25947-0060中加载出来的特征形状，类型： torch.Size([488, 39]) <class 'torch.Tensor'>
从文件1898-145715-0046中加载出来的特征形状，类型： torch.Size([735, 39]) <class 'torch.Tensor'>
从文件3879-173592-0035中加载出来的特征形状，类型： torch.Size([405, 39]) <class '

...

从文件6078-54013-0035中加载出来的特征形状，类型： torch.Size([571, 39]) <class 'torch.Tensor'>
从文件307-127535-0045中加载出来的特征形状，类型： torch.Size([383, 39]) <class 'torch.Tensor'>
从文件8465-246943-0000中加载出来的特征形状，类型： torch.Size([867, 39]) <class 'torch.Tensor'>
从文件1867-148436-0010中加载出来的特征形状，类型： torch.Size([447, 39]) <class 'torch.Tensor'>
从文件4640-19189-0004中加载出来的特征形状，类型： torch.Size([483, 39]) <class 'torch.Tensor'>
从文件8088-284756-0080中加载出来的特征形状，类型： torch.Size([487, 39]) <class 'torch.Tensor'>
从文件150-132655-0031中加载出来的特征形状，类型： torch.Size([228, 39]) <class 'torch.Tensor'>
从文件1455-138263-0007中加载出来的特征形状，类型： torch.Size([914, 39]) <class 'torch.Tensor'>
从文件5322-7679-0010中加载出来的特征形状，类型： torch.Size([499, 39]) <class 'torch.Tensor'>
从文件6147-34605-0021中加载出来的特征形状，类型： torch.Size([630, 39]) <class 'torch.Tensor'>
从文件8629-261139-0064中加载出来的特征形状，类型： torch.Size([322, 39]) <class 'torch.Tensor'>
从文件2436-2477-0015中加载出来的特征形状，类型： torch.Size([780, 39]) <class 'torch.Tensor'>
从文件32-4137-0030中加载出来的特征形状，类型： torch.Size([381, 39]) <class 't

960it [00:01, 890.74it/s]

从文件374-180299-0001中加载出来的特征形状，类型： torch.Size([327, 39]) <class 'torch.Tensor'>
从文件250-142286-0027中加载出来的特征形状，类型： torch.Size([997, 39]) <class 'torch.Tensor'>
从文件5514-19192-0047中加载出来的特征形状，类型： torch.Size([578, 39]) <class 'torch.Tensor'>
从文件911-130578-0013中加载出来的特征形状，类型： torch.Size([639, 39]) <class 'torch.Tensor'>
从文件322-124146-0037中加载出来的特征形状，类型： torch.Size([376, 39]) <class 'torch.Tensor'>
从文件3240-131232-0001中加载出来的特征形状，类型： torch.Size([380, 39]) <class 'torch.Tensor'>
从文件6181-216552-0073中加载出来的特征形状，类型： torch.Size([626, 39]) <class 'torch.Tensor'>
从文件8108-280354-0018中加载出来的特征形状，类型： torch.Size([316, 39]) <class 'torch.Tensor'>
从文件5390-30096-0009中加载出来的特征形状，类型： torch.Size([569, 39]) <class 'torch.Tensor'>
从文件3240-131231-0053中加载出来的特征形状，类型： torch.Size([528, 39]) <class 'torch.Tensor'>
从文件2289-152258-0036中加载出来的特征形状，类型： torch.Size([762, 39]) <class 'torch.Tensor'>
从文件4018-103416-0012中加载出来的特征形状，类型： torch.Size([549, 39]) <class 'torch.Tensor'>
从文件1334-135589-0093中加载出来的特征形状，类型： torch.Size([720, 39]) <c

1078it [00:01, 897.20it/s]

从文件201-127786-0030中加载出来的特征形状，类型： torch.Size([598, 39]) <class 'torch.Tensor'>
从文件8975-270782-0082中加载出来的特征形状，类型： torch.Size([878, 39]) <class 'torch.Tensor'>
从文件2989-138035-0067中加载出来的特征形状，类型： torch.Size([670, 39]) <class 'torch.Tensor'>
从文件3240-131232-0015中加载出来的特征形状，类型： torch.Size([618, 39]) <class 'torch.Tensor'>
从文件1926-147987-0023中加载出来的特征形状，类型： torch.Size([987, 39]) <class 'torch.Tensor'>
从文件5456-24741-0005中加载出来的特征形状，类型： torch.Size([312, 39]) <class 'torch.Tensor'>
从文件1183-128659-0010中加载出来的特征形状，类型： torch.Size([330, 39]) <class 'torch.Tensor'>
从文件4813-248641-0013中加载出来的特征形状，类型： torch.Size([441, 39]) <class 'torch.Tensor'>
从文件5393-19219-0034中加载出来的特征形状，类型： torch.Size([923, 39]) <class 'torch.Tensor'>
从文件254-12312-0010中加载出来的特征形状，类型： torch.Size([959, 39]) <class 'torch.Tensor'>
从文件1235-135884-0036中加载出来的特征形状，类型： torch.Size([460, 39]) <class 'torch.Tensor'>
从文件5393-19219-0033中加载出来的特征形状，类型： torch.Size([580, 39]) <class 'torch.Tensor'>
从文件4195-186236-0026中加载出来的特征形状，类型： torch.Size([272, 39]) <c

# 训练过程
**由于经验比较少，昨日在第一日实现的时候使用了L2正则化，但是一直没有想到L2正则化仅仅是0.0005这么小的参数就造成了  
巨大的影响导致模型甚至经过训练导致在训练集和测试集上的ACC不断变小。现在恢复了正常，再次将L2Reg加上之后看看训练效果：发现  
确实是正则化导致的模型表现型太差了，为什么Loss确实在变小但是Acc变小的太剧烈了，**  
## 实验结果
**未正则化时对于四层神经网络（仅dropout+BatchNormalization）：**
<img src="NoReg.png" alt="NoReg.png" width="600">  
**正则化时Lambda=0.5对于（同上）：**  
<img src="L2Reg_lambda5e-1.png" alt="L2Reg_lambda5e-1.png" width="600">  
**正则化时Lambda=0.00005对于（同上）：**
懒得放图上来了，反正效果差不多。研究下啥原因

In [101]:
if __name__ == "__main__":
    # 你的训练代码
    model = Classifier(input_dim=input_dim*concat_nframes, output_dim=41, hidden_layers=2, hidden_dim=256).to(device)
    best_acc, loss_record, acc_record = train(train_data, dev_data, model, config, device)



True
[001/005] Train Acc: 0.491273 Loss: 1.767532 | Dev Acc: 0.547814 loss: 1.507552
saving model with the acc of 0.548
True
[002/005] Train Acc: 0.540405 Loss: 1.530770 | Dev Acc: 0.567938 loss: 1.420743
saving model with the acc of 0.568
True
[003/005] Train Acc: 0.554932 Loss: 1.469293 | Dev Acc: 0.577353 loss: 1.381545
saving model with the acc of 0.577
True
[004/005] Train Acc: 0.562546 Loss: 1.435526 | Dev Acc: 0.584909 loss: 1.355590
saving model with the acc of 0.585
True
[005/005] Train Acc: 0.568274 Loss: 1.412662 | Dev Acc: 0.587465 loss: 1.341139
saving model with the acc of 0.587
